In [1]:
import sklearn
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pystan as ps

# Tutorial using Boston data

In [2]:
Boston = load_boston()
X = Boston.data
y = Boston.target
# 学習用・検証用にデータを分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [16]:
X

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]])

In [3]:
X_train.shape[0]

404

# stan(data,parameters,modelの3つの情報を渡す必要がある)
# model : $y_i$ ~ $norm(\beta_0 + \beta \boldsymbol[X]^i + \epsilon)$

In [4]:
# データをpythonからStanに渡す。辞書型で変数名を付けて引き渡す。
# X_train.shape[0] = 行数（サンプルサイズ）　X_train.shape[1] = 特徴量＋目的変数の数
dat = {'N': X_train.shape[0], 'M': X_train.shape[1], 'X': X_train, 'y': y_train}

In [5]:
# Stanコード
"""
Stanのコードは大まかに以下の3つの部分に分かれています。

data
データの型などを定義します。配列の長さを明示的に指定（データ数Nや特徴量の数M）するのが地味に大事です。RやPythonを使っていると忘れがちになりますが、StanはC++がベースなので、ちゃんとメモリ領域を人間が指定する必要があります。説明変数が多いときは、matrixで与えるとコードがすっきりします。
parameters
推定するパラメータを定義します。ここも説明変数が多いときは、vectorで係数を与えるとコードがすっきりします。
model
統計モデルを定義します。今回は目的変数がMEDV（y）, 説明変数が他すべてで、残差が正規分布に従うとした場合の正規線形モデルを考えています。
dataやparameterでmatrixやvectorで定義した場合は、各レコードに関して係数とデータの内積（dot_product）をとり、for文で回す、とするとすっきりモデル式が書けます。
"""

"""
記法
行末に;をつける。
コメントアウトは//
"""
model = """
    data {
        int<lower=0> N; //変数N－int型で最小値は0
        int<lower=0> M; //変数M－int型で最小値は0
        matrix[N, M] X; //変数X-matrix型でN行M列である配列
        vector[N] y; //変数y-vector型でN行であるベクトル
    }
    parameters { 
        real beta_0; //変数beta_0-real型_実数型
        vector[M] beta; //変数beta-vector型でM行のベクトル。特徴量M個分のパラメータ
        real<lower=0> sigma; //変数sigma-real型で最小値は0
    }
    model { 
        for (i in 1:N)
            y[i] ~ normal(beta_0 + dot_product(X[i] , beta), sigma);
    }
"""

In [6]:
%time 
stm = ps.StanModel(model_code=model)

# # 計算時間
# CPU times: user 1.11 s, sys: 64 ms, total: 1.18 s
# Wall time: 1min 21s

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_b321a48651c552594cc5636330ed35fa NOW.


Wall time: 0 ns


In [7]:
n_itr = 5000
n_warmup = 1000
chains = 2

# サンプリングの実行
%time fit = stm.sampling(data=dat, iter=n_itr, chains=chains, n_jobs=-1, warmup=n_warmup, algorithm="NUTS", verbose=False)

Wall time: 5min 31s


In [8]:
fit

Inference for Stan model: anon_model_b321a48651c552594cc5636330ed35fa.
2 chains, each with iter=5000; warmup=1000; thin=1; 
post-warmup draws per chain=4000, total post-warmup draws=8000.

            mean se_mean     sd   2.5%    25%     50%    75%   97.5%  n_eff   Rhat
beta_0     40.99     0.1   5.51   30.2  37.29   40.99  44.71   51.88   3243    1.0
beta[1]    -0.12  3.5e-4   0.03  -0.18  -0.14   -0.12   -0.1   -0.06   8854    1.0
beta[2]     0.04  1.9e-4   0.02   0.01   0.03    0.04   0.05    0.07   6675    1.0
beta[3]    -0.03  8.5e-4   0.07  -0.17  -0.08   -0.03   0.02    0.11   6782    1.0
beta[4]      2.8    0.01   1.04   0.75   2.11     2.8   3.49    4.84  10028    1.0
beta[5]   -16.31    0.06    4.2 -24.59 -19.16  -16.32 -13.45   -8.05   5744    1.0
beta[6]     3.21  7.1e-3   0.45   2.34   2.91    3.21   3.51    4.09   4030    1.0
beta[7]  -5.0e-3  1.7e-4   0.01  -0.03  -0.01 -5.1e-3 4.7e-3    0.02   7297    1.0
beta[8]    -1.46  2.8e-3   0.22  -1.89   -1.6   -1.46  -1.31   -

# Tutorial2 space model with local level

In [12]:
# for observed
n_sample = 100
y = np.zeros(n_sample)

# initial param for mu
mu_zero = 100
# mu
mu = np.zeros(n_sample)
#sigma w
s_w = 1000
# sigma v
s_v = 5000

In [14]:
mu[1] = np.random.normal(
loc = mu_zero,
scale = np.sqrt(s_v),
size = 1)

In [15]:
mu

array([  0.        , 201.44955687,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.  